<a href="https://colab.research.google.com/github/mspendyala/aws_bedrock_samples/blob/main/aws_bedrock_example_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install boto3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 4.6 MB/s eta 0:00:00


In [21]:
import os

# Replace with your AWS credentials (use temporary credentials if possible)
os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""
os.environ["AWS_REGION"] = "us-east-1"

print("AWS credentials configured securely in Colab!")

AWS credentials configured securely in Colab!


In [24]:
import boto3
import json
import os
from botocore.exceptions import NoCredentialsError


# Load AWS credentials from Colab environment variables
aws_access_key = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
aws_region = os.getenv("AWS_REGION", "us-east-1")


# Initialize S3 & Bedrock Clients
s3_client = boto3.client(
    "s3",
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key,
    region_name=aws_region,
)

bedrock_client = boto3.client(
    "bedrock-runtime",
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key,
    region_name=aws_region,
)

# S3 Bucket and File
S3_BUCKET = "my-bedrock-knowledge"
S3_FILE_KEY = "knowledge_base.json"

def check_s3_file():
    """Check if the knowledge base file exists in S3"""
    try:
        response = s3_client.list_objects_v2(Bucket=S3_BUCKET)
        if "Contents" in response:
            print("\n✅ Files in S3 bucket:")
            for obj in response["Contents"]:
                print(f"- {obj['Key']}")
                if obj["Key"] == S3_FILE_KEY:
                    return True
        print(f"⚠️ File {S3_FILE_KEY} NOT found in S3 bucket `{S3_BUCKET}`. Upload it first.")
        return False
    except ClientError as e:
        print(f"❌ Error checking S3: {e}")
        return False

def fetch_knowledge_from_s3():
    """Retrieve knowledge data from S3"""
    try:
        response = s3_client.get_object(Bucket=S3_BUCKET, Key=S3_FILE_KEY)
        knowledge_data = json.loads(response["Body"].read().decode("utf-8"))
        return knowledge_data
    except NoCredentialsError:
        print("❌ AWS credentials not found!")
        return []
    except ClientError as e:
        print(f"❌ Error fetching knowledge base: {e}")
        return []

def retrieve_relevant_context(user_query, knowledge_base):
    """Simple keyword-based retrieval from S3 knowledge base"""
    relevant_context = []
    for entry in knowledge_base:
        if user_query.lower() in entry["question"].lower():
            relevant_context.append(entry["answer"])

    return "\n".join(relevant_context) if relevant_context else "No relevant knowledge found."

def query_bedrock(prompt):
    """Query Amazon Bedrock with the correct prompt format for Claude"""
    formatted_prompt = f'\n\nHuman: {prompt}\n\nAssistant:'
    print(f"\nPrompt sent to Bedrock:\n{formatted_prompt}")
    try:
        response = bedrock_client.invoke_model(
            modelId="anthropic.claude-instant-v1",  # Change model if needed
            body=json.dumps({
                "prompt": formatted_prompt,
                "max_tokens_to_sample": 500
            })
        )
        result = json.loads(response["body"].read().decode("utf-8"))
        return result.get("completion", "No response from model")
    except ClientError as e:
        print(f"Error calling Bedrock: {e}")
        return "Error in Bedrock call."


def main():
    """Main workflow for RAG using S3 & Amazon Bedrock"""
    user_query = "What is Retrieval-Augmented Generation?"

    # Step 1: Check if the knowledge base file exists in S3
    if not check_s3_file():
        return

    # Step 2: Fetch knowledge data from S3
    knowledge_base = fetch_knowledge_from_s3()

    # Step 3: Retrieve relevant knowledge
    retrieved_context = retrieve_relevant_context(user_query, knowledge_base)

    # Step 4: Create final prompt for Bedrock
    final_prompt = f"Using the following information:\n{retrieved_context}\n\nAnswer this: {user_query}"

    # Step 5: Query Bedrock with RAG-enhanced prompt
    response_rag = query_bedrock(final_prompt)

    # Step 6: Query Bedrock without retrieval (Vanilla LLM)
    response_vanilla = query_bedrock(user_query)

    print("\n🔹 **Vanilla Bedrock Response:**")
    print(response_vanilla)
    print("\n🔹 **RAG-Enhanced Response (Using S3):**")
    print(response_rag)

if __name__ == "__main__":
    main()





✅ Files in S3 bucket:
- knowledge_base.json

Prompt sent to Bedrock:


Human: Using the following information:
RAG is an AI technique that retrieves relevant knowledge before generating responses to improve accuracy.

Answer this: What is Retrieval-Augmented Generation?

Assistant:

Prompt sent to Bedrock:


Human: What is Retrieval-Augmented Generation?

Assistant:

🔹 **Vanilla Bedrock Response:**
 Retrieval-augmented generation (or retrieve-and-refine) is a technique used in language models where candidate text is retrieved from a large corpus and then refined or modified to better suit the context of the conversation. 

Some key aspects of retrieval-augmented generation include:

- Using a dense semantic search or retrieval model to find relevant text from a large corpus that is similar in meaning to the context of the conversation. This text could be full passages, sentences or snippets.

- Taking the retrieved text as a starting point or "draft" response and modifying it through 